In [15]:
import pandas as pd
from typing import Union, List, Tuple, Optional
pd.options.display.float_format = '{:,.2f}'.format

# Optimizing Memory Use

In [16]:
# sal = pd.read_csv("../data/salary2.csv")

# sal.info()

# df = sal
# df.info(memory_usage='deep')
# df["Department"] = df["Department"].astype("category")
# df["jobTitle"] = df["jobTitle"].astype("category")
# df["Employee_Name"] = df["Employee_Name"].astype("string")
# df.info(memory_usage='deep')

# optimized_dtypes = {
#     "CalYear": "int16",
#     "Employee_Name": "string",
#     "Department": "category",
#     "jobTitle": "category",
#     "Annual_Rate": "float32",
#     "Regular_Rate": "float32",
#     "Overtime_Rate": "float32",
#     "Incentive_Allowance": "float32",
#     "Other": "float32",
#     "YTD_Total": "float32",
#     "ObjectId": "int32"
# }
# df = df.astype(optimized_dtypes)


In [17]:
optimized_dtypes = {
    "CalYear": "int16",
    "Employee_Name": "string",
    "Department": "category",
    "jobTitle": "category",
    "Annual_Rate": "float32",
    "Regular_Rate": "float32",
    "Overtime_Rate": "float32",
    "Incentive_Allowance": "float32",
    "Other": "float32",
    "YTD_Total": "float32",
    "ObjectId": "int32"
}

sal = pd.read_csv("../data/salary2.csv", dtype=optimized_dtypes)


In [18]:
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId
0,2021,<NA>,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,NaN,687.00,1
1,2021,<NA>,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,NaN,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00",NaN,"30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12",NaN,"36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00",NaN,"18,481.53",5


In [19]:
sal["Other"].value_counts()

Series([], Name: count, dtype: int64)

## Cleaning

In [20]:
sal = sal.fillna({
    "Employee_Name": "Unknown",
    "Other": 0
})

sal = sal.drop("Other", axis=1)
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId
0,2021,Unknown,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,687.00,1
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5


In [ ]:
# hr rate and converts to min wage if less than 7.25
sal["Hr_Rate"] = sal["Annual_Rate"] / 2080 
sal["Hr_Rate"] = sal["Hr_Rate"].mask(sal["Hr_Rate"] < 7.25, 7.25)

# cal Overtime rate 
sal["Ot_Rate"] = sal["Hr_Rate"] * 1.5 

# finding and filtering out part time employees
sal['Reg_Hours'] = sal["Annual_Rate"] / sal["Hr_Rate"]
sal = sal[sal['Reg_Hours'] >= 2080]

# calculating overtime hours
sal['Ot_Hours'] = sal["Overtime_Rate"] / sal["Ot_Rate"] 

sal['Total_Hours'] = sal['Reg_Hours'] + sal['Ot_Hours']

sal["Weeks_Worked"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * 40) 

sal["Reg_Hrs_per_Week"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * sal["Weeks_Worked"])

sal["Ot_Hrs_per_Week"] = sal["Overtime_Rate"] / (sal["Ot_Rate"] * sal["Weeks_Worked"])

sal["Avg_Hrs_per_Week"] = sal["Reg_Hrs_per_Week"] + sal["Ot_Hrs_per_Week"]

sal.head(10)

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2,15.00,22.50,"2,080.00",0.00,"2,080.00",1.00,40.00,0.00,40.00
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3,13.67,20.51,"2,080.00",0.00,"2,080.00",51.96,40.00,0.00,40.00
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4,16.95,25.43,"2,080.00",0.00,"2,080.00",52.00,40.00,0.00,40.00
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5,10.30,15.45,"2,080.00",61.66,"2,141.66",40.13,40.00,1.54,41.54
5,2021,"Ammon, Darrell",Louisville Metro Police,Criminal Justice Specialist,"50,107.20","49,362.47",0.00,0.00,"49,362.47",6,24.09,36.14,"2,080.00",0.00,"2,080.00",51.23,40.00,0.00,40.00
6,2021,"Guagliardo, Paul",County Attorney,Senior Attorney,"98,425.60","73,865.12",0.00,0.00,"89,007.52",7,47.32,70.98,"2,080.00",0.00,"2,080.00",39.02,40.00,0.00,40.00
7,2021,"Waggoner, David",Louisville Fire,Fire Prevention Inspector I,"58,963.84","59,557.82",0.00,"13,290.04","72,847.86",8,28.35,42.52,"2,080.00",0.00,"2,080.00",52.52,40.00,0.00,40.00
8,2021,"Clay, Timothy",Louisville Zoo,Zoo Crew Leader-Seasonal,"29,120.00","19,052.80",160.80,0.00,"19,213.60",9,14.00,21.00,"2,080.00",7.66,"2,087.66",34.02,40.00,0.23,40.23
9,2021,"Lenahan, Larry",Finance,Budget Analyst I,"36,972.00","25,963.56",189.81,0.00,"26,153.37",10,17.77,26.66,"2,080.00",7.12,"2,087.12",36.52,40.00,0.19,40.19
10,2021,"Taylor, Steven",Louisville Zoo,Assistant Director,"86,041.80","85,080.97",0.00,0.00,"85,742.83",11,41.37,62.05,"2,080.00",0.00,"2,080.00",51.42,40.00,0.00,40.00


In [22]:
sal = sal.sort_values(by="Avg_Hrs_per_Week", ascending=False)
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
23524,2025,"Leonard, Dillon Michael",Louisville Fire,Firefighter 56hr,"63,441.66",12.04,51.01,0.00,63.05,23525,30.50,45.75,"2,080.00",1.11,"2,081.11",0.01,40.00,112.98,152.98
34985,2024,"Scharrer, Justin",Emergency Management Services,EMT,"60,424.00","5,513.59","18,510.80",0.00,"33,640.39",34986,29.05,43.57,"2,080.00",424.80,"2,504.80",4.74,40.00,89.53,129.53
14300,2023,"Murphy, Paul P.",Louisville Fire,Fire Apparatus Operator 56hr,"72,077.82","40,383.16","97,922.36","39,150.60","177,456.12",14301,34.65,51.98,"2,080.00","1,883.88","3,963.88",29.13,40.00,64.66,104.66
38644,2024,"McKeehan, Matthew Douglas",Emergency Management Services,EMT,"45,926.40","13,663.62","31,838.34",350.00,"45,851.96",38645,22.08,33.12,"2,080.00",961.30,"3,041.30",15.47,40.00,62.14,102.14
15465,2023,"Herndon, Adam B",Louisville Fire,Fire Company Commander 56hr,"82,278.14","42,926.26","94,210.22","29,398.80","167,021.05",15466,39.56,59.34,"2,080.00","1,587.76","3,667.76",27.13,40.00,58.53,98.53


In [23]:
sal['Reg_Hours'].value_counts()

Reg_Hours
2,080.00    39282
Name: count, dtype: int64

In [24]:
def year_selector(
    df: pd.DataFrame,
    year: Union[int, List[int], Tuple[int, int]]
) -> pd.DataFrame:
    """
    Filter a DataFrame by one or more calendar years.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing a 'CalYear' column.
    year : int | list[int] | tuple[int, int]
        The year or years to filter by.
        - If int: returns rows for that year only.
        - If list of ints: returns rows matching any year in the list.
        - If tuple of (start, end): returns rows within that inclusive range.

    Returns
    -------
    pd.DataFrame
        A filtered DataFrame containing only rows that match the given year(s).

    Examples
    --------
    >>> year_selector(sal, 2021)
    # Returns rows where CalYear == 2021

    >>> year_selector(sal, [2020, 2021])
    # Returns rows where CalYear is either 2020 or 2021

    >>> year_selector(sal, (2019, 2021))
    # Returns rows where CalYear is between 2019 and 2021 inclusive
    """
    if isinstance(year, int):
        return df[df["CalYear"] == year]
    elif isinstance(year, list):
        return df[df["CalYear"].isin(year)]
    elif isinstance(year, tuple) and len(year) == 2:
        start, end = year
        return df[(df["CalYear"] >= start) & (df["CalYear"] <= end)]
    else:
        raise TypeError("`year` must be an int, list of ints, or tuple of (start, end).")
    return df

In [25]:
sal_2025 = year_selector(sal, 2025)
sal_2025

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
23524,2025,"Leonard, Dillon Michael",Louisville Fire,Firefighter 56hr,"63,441.66",12.04,51.01,0.00,63.05,23525,30.50,45.75,"2,080.00",1.11,"2,081.11",0.01,40.00,112.98,152.98
21009,2025,"Fenwick, Paul E",Louisville Metro Police Department,Police Officer,"95,555.20","75,315.67","134,672.06","10,093.88","220,081.61",21010,45.94,68.91,"2,080.00","1,954.32","4,034.32",40.99,40.00,47.68,87.68
21560,2025,"Standard, Royce Leshawn",Department of Corrections,Corrections Officer,"69,596.80","55,491.41","95,985.71",0.00,"151,477.12",21561,33.46,50.19,"2,080.00","1,912.45","3,992.45",41.46,40.00,46.13,86.13
22397,2025,"Stimphil, Richardson",Department of Corrections,Corrections Officer,"66,331.20","50,618.52","86,647.31",0.00,"137,265.83",22398,31.89,47.84,"2,080.00","1,811.38","3,891.38",39.68,40.00,45.65,85.65
26842,2025,"Mann, Mikayla Faith",Emergency Management Services,EMS EMT-Paramedic I,"62,046.40","3,740.70","6,106.35",0.00,"9,847.05",26843,29.83,44.74,"2,080.00",136.47,"2,216.47",3.14,40.00,43.53,83.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27302,2025,"Cheatham, Paulette D",Louisville Zoo,Receptionist ZU,"36,379.20",0.00,0.00,0.00,0.00,27303,17.49,26.24,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
27436,2025,"Washington, Kingston",Parks & Recreation,Parks & Rec. Aide,"34,632.00",0.00,0.00,0.00,0.00,27437,16.65,24.97,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
27472,2025,"Fields, Christian",Parks & Recreation,Parks & Rec. Aide,"34,632.00",0.00,0.00,0.00,0.00,27473,16.65,24.97,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
27550,2025,"Wescott, Brenda",Louisville Metro Police Department,Police Officer,"63,585.60",0.00,0.00,0.00,0.00,27551,30.57,45.86,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN


In [26]:
def department_filter(
    df: pd.DataFrame,
    department: Union[str, List[str]]
) -> pd.DataFrame:
    """
    Filter a DataFrame by one or more department names.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing a 'Department' column.
    department : str | list[str]
        Department name(s) to filter by.
        - If str: returns rows for that department only.
        - If list of str: returns rows matching any department in the list.

    Returns
    -------
    pd.DataFrame
        A filtered DataFrame containing only rows for the given department(s).

    Examples
    --------
    >>> department_filter(sal, "Louisville Metro Police Department")
    # Returns rows for that department

    >>> department_filter(sal, ["Parks & Recreation", "Library"])
    # Returns rows for both departments
    """
    # Clean column names once to avoid hidden tab or whitespace issues
    df.columns = df.columns.str.strip()

    if isinstance(department, str):
        return df[df["Department"] == department]
    elif isinstance(department, list):
        return df[df["Department"].isin(department)]
    else:
        raise TypeError("`department` must be a string or list of strings.")


In [27]:
lmpd25 = department_filter(sal_2025,"Louisville Metro Police Department")
lmpd25

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
21009,2025,"Fenwick, Paul E",Louisville Metro Police Department,Police Officer,"95,555.20","75,315.67","134,672.06","10,093.88","220,081.61",21010,45.94,68.91,"2,080.00","1,954.32","4,034.32",40.99,40.00,47.68,87.68
25482,2025,"Richardson, Joshua Thomas",Louisville Metro Police Department,Police Officer,"78,790.40","58,987.21","76,854.67","8,280.70","144,122.58",25483,37.88,56.82,"2,080.00","1,352.60","3,432.60",38.93,40.00,34.74,74.74
23252,2025,"Aviles, Celestino Ardres Ortiz",Louisville Metro Police Department,Police Officer,"80,246.40","63,823.73","82,673.52","8,323.57","154,820.81",23253,38.58,57.87,"2,080.00","1,428.61","3,508.61",41.36,40.00,34.54,74.54
21730,2025,"Williams, Derrick D",Louisville Metro Police Department,Police Officer,"90,584.00","71,934.12","87,857.76","7,954.99","167,746.88",21731,43.55,65.32,"2,080.00","1,344.93","3,424.93",41.29,40.00,32.57,72.57
21005,2025,"Cadwell, Corey R",Louisville Metro Police Department,Police Sergeant,"111,550.40","89,179.76","100,464.80","7,711.35","197,355.91",21006,53.63,80.44,"2,080.00","1,248.86","3,328.86",41.57,40.00,30.04,70.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25294,2025,"Heady, David Ellis",Louisville Metro Police Department,Storage Equipment Operator5day,"42,452.80",0.00,0.00,0.00,"10,658.08",25295,20.41,30.61,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
25301,2025,"Kizer, Larry W",Louisville Metro Police Department,Storage Equipment Operator5day,"42,452.80",0.00,0.00,0.00,0.00,25302,20.41,30.61,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
25514,2025,"Helm, Brent Michael",Louisville Metro Police Department,Police Recruit,"47,673.60",0.00,0.00,0.00,0.00,25515,22.92,34.38,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
27027,2025,"Price, Nichole",Louisville Metro Police Department,Information Process Technician I,"39,769.60",0.00,0.00,0.00,0.00,27028,19.12,28.68,"2,080.00",0.00,"2,080.00",0.00,NaN,NaN,NaN
